In [8]:
import dask.dataframe as dd
import geopandas as gpd
import dask_geopandas as dgpd

In [15]:
df = dd.read_parquet(
    's3://overturemaps-us-west-2/release/2023-07-26-alpha.0/theme=places/type=place/*',
    # columns=['bbox', 'geometry'],
    engine='pyarrow',
    index='id',
    dtype_backend='pyarrow',
    storage_options={"anon": True},
    parquet_file_extensions=False,
)

In [19]:
from overturemapsdownloader.utils import read_geospatial_data
bbox_filter = read_geospatial_data('bbox.geojson', as_shapely_str=True, output_format='Custom')
print(bbox_filter)

POLYGON ((31.53149079200861 30.101963360401314, 31.428791618303393 30.101963360401314, 31.428791618303393 29.997741213406755, 31.53149079200861 29.997741213406755, 31.53149079200861 30.101963360401314))


/Users/yharby/Library/Caches/pypoetry/virtualenvs/overturemapsdownloader-W8Mf4UPw-py3.11/lib/python3.11/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


In [20]:
geometry = df["geometry"].map_partitions(gpd.GeoSeries.from_wkb, meta=gpd.GeoSeries(name="geometry")).set_crs(4326)
gdf = dgpd.from_dask_dataframe(df, geometry=geometry)

clipped_gdf = gdf[gdf.geometry.within(bbox_filter)]
print(clipped_gdf.head())

                                                   updatetime  version  \
id                                                                       
tmp_923D8A13EE34D545D4C50D605CBD720D  2023-07-24T00:00:00.000        0   
tmp_349425DB28AC143C183EAD86B2A5C170  2023-07-24T00:00:00.000        0   
tmp_42028412FDC41BC2067899F403FF20BF  2023-07-24T00:00:00.000        0   
tmp_BFCBD5CAB864C656A010B86B7334257C  2023-07-24T00:00:00.000        0   
tmp_052B14C9CA6F13AE00870F7C3FC4735A  2023-07-24T00:00:00.000        0   

                                                                                  names  \
id                                                                                        
tmp_923D8A13EE34D545D4C50D605CBD720D  [('common', array([list([('value', 'Real Estat...   
tmp_349425DB28AC143C183EAD86B2A5C170  [('common', array([list([('value', 'عيادات اد....   
tmp_42028412FDC41BC2067899F403FF20BF  [('common', array([list([('value', 'الصفحة الر...   
tmp_BFCBD5CAB864C656A010B8

In [21]:
clipped_gdf.to_parquet('output.parquet')